[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()


# Financial Analysis

본 노트북은 [Live Corona Map](https://github.com/LiveCoronaDetector/livecod/) 프로젝트에 향후 추가할 계획인 금융 분석 시각화를 위해 만들어졌습니다.


# Setup

필요한 모듈을 임포트해옵니다. (프로젝트의 진행에 따라 추가, 변경될 수 있습니다.)

In [0]:
!pip install --quiet yfinance

In [0]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import time